# TUnfoldDensity example with pyroot

In [1]:
import ROOT
import numpy as np
import array as array
import math
import matplotlib.pyplot as plt
#import statistics as st
ROOT.gStyle.SetOptStat(000000)

Welcome to JupyROOT 6.20/02


In [2]:
ROOT.gInterpreter.ProcessLine('#include "MyTUnfoldDensity.h"')

0

In [3]:
#%jsroot

## Construct the Binning scheme

Set the axis ranges for the generator (m bins) and detector  (n bins) level distributions as well as the number of bins in each. *Note* that we are currently using twice as many detector bins as generator level bins as recommended by the TUnfold documenation. This condition $n ≥ m$ 
is in contrast to many common unfoldding algorithms which require $n = m$ however a notable example of an unfolding algorithm without that restriction is __[G. D’Agostini](arXiv:1010.0632)__.

In [4]:
### Set up the binning scheme
mbinsGen  = array.array('d',  [0., 50., 100. ])
mbinsDet  = array.array('d', [0., 25.,50.,75.,100. ])
ptbinsGen = array.array('d', [0., 50., 100. ])
ptbinsDet = array.array('d', [0., 50., 100. ])

#mbinsGen  = array.array('d', [0., 25.,50.,75.,100. ])# [0., 50., 100. ])
#mbinsDet  = array.array('d', [0., 12.5, 25.,37.5,50., 62.5,75., 87.5, 100. ])#[0., 25.,50.,75.,100. ])
#ptbinsGen = array.array('d', [0., 25.,50.,75.,100. ])#[0., 50., 100. ])
#ptbinsDet = array.array('d', [0., 25.,50.,75.,100. ])#[0., 50., 100. ])


nmbinsGen  = len(mbinsGen) -1
nmbinsDet  = len(mbinsDet) -1
nptbinsGen = len(ptbinsGen) -1
nptbinsDet = len(ptbinsDet) -1

print("There are {} Gen Mass bins , {} Gen Pt bins ,{} Reco Mass bins , {} Reco Pt bins  ".format(nmbinsGen,nptbinsGen , nmbinsDet,nptbinsDet) )

There are 2 Gen Mass bins , 2 Gen Pt bins ,4 Reco Mass bins , 2 Reco Pt bins  


Instantiate the TUnfoldBinning Scheme. The "signal" node represents the generator level (coarser) binning whereas the "detector" node represents the (finer) input binning scheme. 

In [5]:
generatorBinning = ROOT.TUnfoldBinning("generator")
### Need coarser binning for signal
signalBinning = generatorBinning.AddBinning("signal")    
signalBinning.AddAxis("massgen",nmbinsGen,mbinsGen,
                        True, # needed for fakes
                        True # overflow bin
                        )
signalBinning.AddAxis("ptgen",nptbinsGen,ptbinsGen,
                        True, # needed for fakes
                        True # overflow bin
                        )

# X axis : generator binning is Signal : mgen * ptgen and Background : mrec * ptrec

xaxis =  array.array('d',[a for a in range(nmbinsGen *nptbinsGen + 1 )])
nx = len(xaxis)-1

print (xaxis)
print("There are {} (signal) generator level bins in the simple scheme".format(nx))
print("---------------------------------------------------------------")

print("signalBinning")
for i in range(len(signalBinning.GetDistributionBinning(0))):
    print (signalBinning.GetDistributionBinning(0)[i])

#print "detectorBinning"
#for i in xrange(len(backgroundBinning.GetDistributionBinning(0))):
#    print backgroundBinning.GetDistributionBinning(0)[i]
    
#print "Below counts include overflow and underflow bins"    
#print "There are {} signal {} background bins in the generator scheme".format(signalBinning.GetTH1xNumberOfBins() , backgroundBinning.GetDistributionNumberOfBins())
genBin = generatorBinning

array('d', [0.0, 1.0, 2.0, 3.0, 4.0])
There are 4 (signal) generator level bins in the simple scheme
---------------------------------------------------------------
signalBinning
0.0
50.0
100.0


In [6]:
detectorBinning = ROOT.TUnfoldBinning("detector") 
detectorDistribution=detectorBinning.AddBinning("detectordistribution")
detectorDistribution.AddAxis("mass",nmbinsDet,mbinsDet,
                        False, # no underflow bin (not reconstructed)
                        True # overflow bin
                        )
detectorDistribution.AddAxis("pt",nptbinsDet,ptbinsDet,
                        False, # no underflow bin (not reconstructed)
                        True # overflow bin
                        )

yaxis =  array.array('d',[a for a in range(nmbinsDet *nptbinsDet+1 )])
ny = len(yaxis)-1

print (yaxis)
print ("There are {}  detector level bins in the simple scheme".format(ny))
print ("detectorBinning")
for i in range(len(detectorDistribution.GetDistributionBinning(0))):
    print (detectorDistribution.GetDistributionBinning(0)[i])
    
print ("Below counts include overflow and underflow bins")
print ("There are {} bins in the detector scheme".format(detectorDistribution.GetTH1xNumberOfBins()))#GetDistributionNumberOfBins())
detBin  = detectorBinning

array('d', [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0])
There are 8  detector level bins in the simple scheme
detectorBinning
0.0
25.0
50.0
75.0
100.0
Below counts include overflow and underflow bins
There are 15 bins in the detector scheme


# Create Response Matrix and 1D input 

In [7]:


M = ROOT.TUnfoldBinning.CreateHistogramOfMigrations(generatorBinning,detectorBinning,"M")
M1 = ROOT.TUnfoldBinning.CreateHistogramOfMigrations(generatorBinning,detectorBinning,"M+1#sigma")
M2 = ROOT.TUnfoldBinning.CreateHistogramOfMigrations(generatorBinning,detectorBinning,"M-1#sigma")

h = detectorBinning.CreateHistogram("h")

hup = detectorBinning.CreateHistogram("hup")
hdn = detectorBinning.CreateHistogram("hdn")



htrue = generatorBinning.CreateHistogram("htrue")
htruef = detectorBinning.CreateHistogram("htruef")



/usr/local/bin/start_ipykernel.py:7: RuntimeWarning: creating converter for unknown type "int**"


## Generate the response matrix

Here, we have a respone matrix with 4 coarse and 8 fine bins (configurable). The variable of interest ($x$) ranges from 0 to 100. We assume the resolution on $x$ is 1.0, and that there is a systematic uncertainty "scale", which has an "up" and "down" variation of 2%. We generate 10000 pseudo-events for each value of x between 0 and 100, smeared by the resolution. For each of the "x" values, a reconstructed "y" value is constructed, again smeared by the resolution. We then add the "up" and "down" scale uncertainites. Finally, we construct TH2 matrices that represent the response matrices. 

In [8]:
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)

In [9]:


nev = 100000 # 100000
xavg = 110.
yavg = 110.
xres = 1.
yres = 1.
gen2x = np.random.random(nev) * xavg
reco2x = np.random.randn(nev) * xres + gen2x
gen2y = np.random.random(nev) * yavg
reco2y = np.random.randn(nev) * yres + gen2y
#print mean(reco2y)

error_scaleUp = 1.10
error_scaleDn = 0.90

#gen2yu = np.random.random(nev) * yavg*scaleup
#reco2yu = np.random.randn(nev) * yres + gen2yu
reco2yu = error_scaleUp * reco2y
#print mean(reco2yu)

#gen2yd = np.random.random(nev) * yavg*scaledn
#recoreco2yu = -0.01 * reco2y
reco2yd = error_scaleDn* reco2y

#print mean(reco2yd)

#print reco2yd/reco2y
#reco2yu = np.random.randn(reco2y) * yres

gen2 = np.array( [gen2x,gen2y] ).T
reco2 = np.array( [reco2x,reco2y] ).T
r = np.array( [gen2x, gen2y, reco2x, reco2y] ).T

ru = np.array( [gen2x, gen2y, reco2x, reco2yu] ).T
rd = np.array( [gen2x, gen2y, reco2x, reco2yd] ).T



In [10]:




for i in range(nev):
    xg = r[i,0]
    yg = r[i,1]
    xr = r[i,2]
    yr = r[i,3]
    
    yru = ru[i,3]
    yrd = rd[i,3]
    
    
    glob_recb= detectorDistribution.GetGlobalBinNumber(xr,yr)
    glob_genb= signalBinning.GetGlobalBinNumber(xg, yg)


    h.Fill(glob_recb)

    glob_recbU = detectorDistribution.GetGlobalBinNumber(xr , yru)
    glob_recbD = detectorDistribution.GetGlobalBinNumber(xr , yrd)
      
    M.Fill(glob_genb, glob_recb)#, #1./Npe)
    M1.Fill(glob_genb, glob_recbU)#, 1./Npe)
    M2.Fill(glob_genb, glob_recbD)#, 1./Npe)
    
    hup.Fill(glob_recbU)
    hdn.Fill(glob_recbD)

    
    
    htrue.Fill(glob_genb)
    glob_recb_GenObj = detectorDistribution.GetGlobalBinNumber(xg,yg)

    htruef.Fill(glob_recb_GenObj)
    
   

In [11]:
'''
for bx in xrange(M.GetNbinsX()):
    for by in xrange(M.GetNbinsY()):
        bc = M.GetBinContent(bx,by)
        if bc > 1. :
            gxn = signalBinning.GetBinName(bx)
            gyn = detectorDistribution.GetBinName(by)
            print "Bin X {} {} Y {} {} is filled with {} ".format(bx, gxn, by, gyn, bc)
'''            

'\nfor bx in xrange(M.GetNbinsX()):\n    for by in xrange(M.GetNbinsY()):\n        bc = M.GetBinContent(bx,by)\n        if bc > 1. :\n            gxn = signalBinning.GetBinName(bx)\n            gyn = detectorDistribution.GetBinName(by)\n            print "Bin X {} {} Y {} {} is filled with {} ".format(bx, gxn, by, gyn, bc)\n'

# Normalize by Pt (y) bin

In [12]:
normalize = False

In [13]:
ptbinsgenglobal = [ [0,5] ,[6,8], [10,12] , [13,13]  ]

ptbinsdetglobal = [  [0,0] , [1,5] , [6,10] , [11,15] ]#, [16,18 ]  ]


In [14]:

def NormM(rMMx = None  , recoorgen2 = None,  nameg = None) :

    ### Make a new empty TH2 so the OFL and UFL will be zero
    rMMnx = rMMx.Clone(nameg)
    for xx in range(rMMnx.GetNbinsX()+1) :
        for yy in range(rMMnx.GetNbinsY()+1) :
            rMMnx.SetBinContent(xx , yy ,0.0) 
            rMMnx.SetBinError(xx , yy ,0.0)   
        
    
    intscales3 = [ ]
    intsums = [ ]
    intsums2 = [ ]
    bins1 = [ ]
    bins2 = [ ]

    #if intscalesin == None :
    ptbs1 = None
    ptbs2 = None    
    if recoorgen2 == 'reco':  
        ptbs1 = ptbinsdetglobal
        ptbs2 = ptbinsgenglobal 
        bins1 = detBin
        noden1 = "detectordistribution"
        bins2 = genBin
        noden2 = "signal"
    elif recoorgen2 == 'gen':  
        ptbs1 = ptbinsgenglobal
        ptbs2 = ptbinsdetglobal 
        bins1 = genBin
        noden1 = "signal"
        bins2 = detBin
        noden2 = "detectordistribution"
        
    for iptb,ptbg in enumerate(ptbs1[1:-1]) :
        sumn = 0
        tint = 0.
        #print ptbinsGen[iptb]
        #print ptbinsGen[iptb+1]
        #ptw = ptbinsGen[iptb+1] - ptbinsGen[iptb]
        #print "New Pt bin {}- {} GeV of width {} GeV".format(ptbinsGen[iptb] , ptbinsGen[iptb+1] , ptw )
        for iy in range( ptbg[0], ptbg[1]): #  
            
            db = bins1.FindNode(noden1).GetBinName(iy)
            print (db)
            dbs = bins1.FindNode(noden1).GetBinSize(iy)
            tgbs = dbs #1.     
            bw = 1.
            #print ptw
            #if ptw > 0.:
            #    tgbs = gbs/ptw
            #print    tgbs 
            for iptbr,ptbr in enumerate(ptbs2[1:-1]) :
                for ix in range(ptbr[0],ptbr[1]):
                    print (ptbr[0])
                    print ("to")
                    print (ptbr[1])
                    gb = bins2.FindNode(noden2).GetBinName(ix)
                    #db = detBin.FindNode("detectordistribution").GetBinName(ix)
                    #if 'mass[ofl]' in str(db): 
                    print ("...."+str(gb))
                    #    continue
                    gbs = bins2.FindNode(noden2).GetBinSize(ix)
                    
                    #if iy ==1:print dbs
                    #tdbs = dbs#/ptw
                    #print tdbs
                    bw = gbs*dbs

                    rbc = rMMx.GetBinContent(ix , iy)
                    trbc = rMMx.GetBinContent(ix , iy)
                    #print trbc

                    rbe = rMMx.GetBinError(ix , iy)
                    trbe = rMMx.GetBinError(ix , iy)
                    #print bw
                    if bw > 0. :
                        trbc = rbc/bw
                        trbe = rbe/bw               
                    rMMnx.SetBinContent(ix, iy , trbc)    
                    rMMnx.SetBinError(ix, iy , trbe)  
                    tint+=trbc
                    sumn+=1
        print ("Integral of bin {} {} is {} and it was the sum of {} bins".format(ptbg, iptb, tint, sumn)     )   
        intscales3.append(tint)
        intsums.append(sumn)
    print ("Integral of bin x :")
    print (intscales3)
    #elif intscalesin != None :
    #    intscales = tint #intscalesin
    print ("scaling by {}".format(tint))
    for iptb,ptbg in enumerate(ptbs1[1:-1]) :
        sumn2 = 0
        sdsdx = intscales3[iptb]
        #print sdsdx
        #print ptbg
        #print iptb
        #print "New Pt bin "
        for iy in range( ptbg[0], ptbg[1]): #  
            for iptbr,ptbr in enumerate(ptbs2[1:-1]) :
                for ix in range(ptbr[0],ptbr[1]):

                    rbc = rMMnx.GetBinContent(ix , iy)
                    trbc = rMMnx.GetBinContent(ix , iy)
                    #print trbc

                    rbe = rMMnx.GetBinError(ix , iy)
                    trbe = rMMnx.GetBinError(ix , iy)
                    #print bw
                    if sdsdx > 0. :
                        trbc = rbc/sdsdx
                        trbe = rbe/sdsdx               
                    rMMnx.SetBinContent(ix , iy , trbc)    
                    rMMnx.SetBinError(ix , iy, trbe) 
                    sumn2 +=1
        intsums2.append(sumn2)     
    print (intsums)
    print (intsums2)    
    return rMMnx #, intscales3 ]



In [15]:
def NormXorY(XorY = None , recorgen = None , nameg = None) :
    ### Make a new empty TH1 so the OFL and UFL will be zero
    XorYn = XorY.Clone()
    for xx in range(XorY.GetNbinsX()+1) :
        XorYn.SetBinContent(xx  ,0.0) 
        XorYn.SetBinError(xx  ,0.0)   
        
    gpts = None
    bins = None
    noden = None

    
    
    if recorgen == 'reco' :
        gpts = ptbinsdetglobal
        bins = detBin
        noden = "detectordistribution"
    elif recorgen == 'gen' :
        gpts = ptbinsgenglobal        
        bins = genBin
        noden = "signal"
    intscalesx = [ ]
    
    for iptbr,ptbr in enumerate(gpts[1:-1]) :
        tint = 0.
        print (iptbr)
        print (ptbr)        
        #print ptbinsGen[iptbr+1]
        #print ptbinsGen[iptbr]        
        #ptw = ptbinsGen[iptbr+1] - ptbinsGen[iptbr]
        for iy in range(ptbr[0],ptbr[1]):
            #print ptbr[0]
            #print "to"
            #print ptbr[1]
            db = bins.FindNode(noden).GetBinName(iy)
            #if 'mass[ofl]' in str(db): 
            print (db)
            #    continue
            dbs = bins.FindNode(noden).GetBinSize(iy)
            #if iy ==1:print dbs
            tdbs = dbs#/ptw
            #if bins == genBin:
            #    tdbs = tdbs * ptw
            bw = tdbs# *ptw

            rbc = XorY.GetBinContent(iy)
            trbc = 0.#XorY.GetBinContent( iy)
            #print trbc

            rbe = XorY.GetBinError( iy)
            trbe = 0.#XorY.GetBinError( iy)
            #print bw
            if bw > 0. :
                trbc = rbc/bw
                trbe = rbe/bw               
            XorYn.SetBinContent(iy , trbc)    
            XorYn.SetBinError(iy , trbe)  
            tint+=trbc
        intscalesx.append(tint)
    print ("Integral of bin x for NormXorY is :")
    print (intscalesx)


    for iptb,ptbg in enumerate(gpts[1:-1]) :
        tint = 0.
        sdsd = intscalesx[iptb]
        print (sdsd)
        print (ptbg)
        print (iptb)
        print ("New Pt bin ")
        for ix in range( ptbg[0], ptbg[1]): #  

            rbc = XorYn.GetBinContent(ix)
            trbc = 0.#XorYn.GetBinContent(ix)
            #print trbc

            rbe = XorYn.GetBinError(ix)
            trbe =0. #XorYn.GetBinError(ix)
            #print bw
            if sdsd > 0. :
                trbc = rbc/sdsd
                trbe = rbe/sdsd                
            XorYn.SetBinContent(ix , trbc)    
            XorYn.SetBinError(ix, trbe)  
    return XorYn.Clone(nameg)

               

In [16]:

respN = 'reco'


if normalize :
    M = NormM(M.Clone() , respN,"M_norm") 


In [17]:
if normalize :
    M1 = NormM(M1.Clone() ,respN , "M_Up") 
    M2 = NormM(M2.Clone() , respN,"M_Dn") 

    h = NormXorY(h.Clone() ,'reco', "h_norm") 

    hup = NormXorY(hup.Clone() ,'reco', "hu_norm") 
    hdn = NormXorY(hdn.Clone() ,'reco', "hd_norm") 

    htrue = NormXorY(htrue.Clone() ,'gen', "htrue_norm") 
    htruef = NormXorY(htruef.Clone() ,'reco', "htruef_norm") 


## Draw the response matrix

We draw the nominal and $\pm 1 \sigma$ variations. 

In [18]:

c0 = ROOT.TCanvas("c0", "c0", 1800, 600)
c0.Divide(3,1)
c0.cd(1)
M.Draw("colz")
c0.cd(2)
M1.Draw("colz")
c0.cd(3)
M2.Draw("colz")
c0.Draw()

## Draw the response matrix ratio

We draw the ratio of the $\pm 1 \sigma$ variations to the nominal. 

In [19]:
M1ratio = M1.Clone()
M1ratio.Divide(M)
M2ratio = M2.Clone()
M2ratio.Divide(M)
M1ratio.SetMaximum(1.2)
M2ratio.SetMaximum(1.2)

cratio = ROOT.TCanvas("cratio", "cratio", 1800, 600)
cratio.Divide(3,1)
cratio.cd(1)
M.Draw("colz")
cratio.cd(2)
M1ratio.Draw("colz")
ROOT.gPad.SetLogz()
cratio.cd(3)
M2ratio.Draw("colz")
ROOT.gPad.SetLogz()
cratio.Draw()

# Project Response Matrix onto X and Y axis 

In [20]:
h2 = M.ProjectionY("reco2")
htrue2 = M.ProjectionX("gen2")



## Plot the reconstructed and truth

In [21]:
crec1 = ROOT.TCanvas("cr1", "creco", 600, 600)

htruef.SetLineColor(2)
h.SetMarkerStyle(20)
h.SetMarkerColor(ROOT.kBlue)
h2.SetMarkerStyle(22)
htruef.SetMarkerStyle(21)
htrue.SetMarkerStyle(23)
htrue2.SetMarkerStyle(24)


h2.SetMarkerColor(ROOT.kMagenta)

htruef.SetMarkerColor(ROOT.kRed)
htrue2.SetMarkerColor(ROOT.kRed)

h.SetLineColor(ROOT.kBlue)
htruef.SetLineColor(ROOT.kRed)
htrue.SetLineColor(ROOT.kBlack)
#htrue.SetLineColor(ROOT.kRed)

h.Draw('e ')


if normalize :
    h.SetMinimum(.15)
    h.SetMaximum(.6)
else:
    h.SetMinimum(.0)
    h.SetMaximum(25000.)
#h.SetMinimum(.0)
#h.SetMaximum(2.)
h2.Draw('e same')


#h2U.Draw('e ')
#h.Draw('e same')
#hup.Draw("hist same ")
#hdn.Draw("hist same")
htruef.Draw("e same")
htrue.Draw("e same")
htrue2.Draw("e same")


leg1 = ROOT.TLegend(0.7,0.7,0.86, 0.86)
leg1.SetBorderSize(0)
leg1.SetFillColor(0)
leg1.AddEntry( h2, 'Reco from M', 'p')
leg1.AddEntry( h, 'Reco', 'p')
#leg1.AddEntry( hup, '+1#sigma', 'l')
#leg1.AddEntry( hdn, '-1#sigma', 'l')
leg1.AddEntry( htruef, 'True', 'p')
leg1.AddEntry( htrue, 'True Gen Binned', 'p')
leg1.AddEntry( htrue2, 'True from M', 'p')

leg1.Draw()
crec1.Draw()

# Check that the matrix projections match the 1D input

In [22]:

h2U = M1.ProjectionY("reco2U")
h2D = M2.ProjectionY("reco2D")


In [23]:
crecp = ROOT.TCanvas("crp", "", 600, 600)

hc = h.Clone("hc")
hc.Divide(h2.Clone())


hcu = hup.Clone("hcU")
hcu.Divide(h2U.Clone())

hcd = hdn.Clone("hcD")
hcd.Divide(h2D.Clone())

hc.SetTitle("Reco/ Reco from M")


hc.SetMarkerStyle(23)
hc.SetMarkerColor(ROOT.kAzure+1)

hcd.SetMarkerStyle(25)
hcd.SetMarkerColor(ROOT.kAzure-5)


hcu.SetMarkerStyle(26)
hcu.SetMarkerColor(ROOT.kAzure+6)


hc.Draw('e  ')
hcu.Draw('e same ')
hcd.Draw('e same ')
hc.SetMinimum(-.5)
hc.SetMaximum(2.5)


leg12p = ROOT.TLegend(0.7,0.7,0.86, 0.86)
leg12p.SetBorderSize(0)
leg12p.SetFillColor(0)

leg12p.AddEntry( hc, 'Nominal', 'p')
leg12p.AddEntry( hcu, '+1#sigma', 'p')
leg12p.AddEntry( hcd, '-1#sigma', 'p')

leg12p.Draw()

crecp.Draw()

# Plot the Up and Down Variations to Reco

In [24]:
crec2 = ROOT.TCanvas("cr2", "", 600, 600)

h.SetMarkerStyle(20)
h.SetMarkerColor(ROOT.kBlue)

hup.SetLineStyle(2)
hdn.SetLineStyle(3)
hup.SetLineColor(ROOT.kBlue)
hdn.SetLineColor(ROOT.kBlue)

#htrue2U = M1.ProjectionX("gen2U")
h2U.SetMarkerStyle(23)
h2U.SetMarkerColor(ROOT.kAzure+5)

h2D.SetMarkerStyle(25)
h2D.SetMarkerColor(ROOT.kAzure-5)



h2U.Draw('e ')
if normalize :
    h2U.SetMinimum(.24)
    h2U.SetMaximum(.26)
else :
    h2U.SetMinimum(.0)
    h2U.SetMaximum(14000)
hup.Draw("hist same")


hdn.Draw("hist same ")



h2D.Draw('e same ')
h.Draw('e same ')


leg12 = ROOT.TLegend(0.7,0.7,0.86, 0.86)
leg12.SetBorderSize(0)
leg12.SetFillColor(0)

leg12.AddEntry( h, 'Reco', 'p')
leg12.AddEntry( hup, '+1#sigma', 'l')
leg12.AddEntry( hdn, '-1#sigma', 'l')
leg12.AddEntry( h2U, '+1#sigma from M1', 'p')
leg12.AddEntry( h2D, '-1#sigma from M2', 'p')

leg12.Draw()

crec2.Draw()

## Unfold

1. Create the `TUnfoldDensity` object. 
1. Add the systematic uncertainty variations via `AddSysError`. 
1. Perform the regularization by scanning the L curve. 
1. Get the output. **Note! This does not account for systematic uncertainties automatically. They must be added by hand.**
1. Draw unfolded versus true distributions (in the same coarse binning scheme). 

In [25]:
orientation = ROOT.TUnfold.kHistMapOutputHoriz
regMode = ROOT.TUnfold.kRegModeCurvature
con = ROOT.TUnfold.kEConstraintArea #ROOT.TUnfold.kEConstraintArea
mode =  ROOT.TUnfoldDensity.kDensityModeBinWidth
axisSteering =  "*[b]"

#u = ROOT.TUnfoldDensity(M, orientation, regMode, con, mode, genBin, detBin, "signal", axisSteering)

u = ROOT.TUnfoldDensity(M,orientation,regMode,con)


Info in <TUnfold::SetConstraint>: fConstraint=1
Warning in <TUnfold::TUnfold>: 9 output bins do not depend on the input data  0-5 9 13 17
Info in <TUnfold::TUnfold>: 15 input bins and 9 output bins
Info in <TUnfoldDensity::TUnfold>: *NOT* unfolding bin #0 (xaxis:generator[ufl])
Info in <TUnfoldDensity::TUnfold>: *NOT* unfolding bin #1 (xaxis:generator[0.5,1.5])
Info in <TUnfoldDensity::TUnfold>: *NOT* unfolding bin #2 (xaxis:generator[1.5,2.5])
Info in <TUnfoldDensity::TUnfold>: *NOT* unfolding bin #3 (xaxis:generator[2.5,3.5])
Info in <TUnfoldDensity::TUnfold>: *NOT* unfolding bin #4 (xaxis:generator[3.5,4.5])
Info in <TUnfoldDensity::TUnfold>: *NOT* unfolding bin #5 (xaxis:generator[4.5,5.5])
Info in <TUnfoldDensity::TUnfold>: *NOT* unfolding bin #9 (xaxis:generator[8.5,9.5])
Info in <TUnfoldDensity::TUnfold>: *NOT* unfolding bin #13 (xaxis:generator[12.5,13.5])
Info in <TUnfoldDensity::TUnfold>: *NOT* unfolding bin #17 (xaxis:generator[ofl])
Info in <TUnfoldDensity::RegularizeOneDis

In [26]:
u.SetInput( h )


0

In [27]:

u.AddSysError(M1, "Up", orientation, ROOT.TUnfoldDensity.kSysErrModeMatrix)


In [28]:

u.AddSysError(M2, "Dn", orientation, ROOT.TUnfoldDensity.kSysErrModeMatrix)


In [29]:

u.DoUnfold(0.0)

0.059947656927504006

In [30]:
# Get output... DOES NOT CONTAIN SYSTEMATICS. 
o = u.GetOutput("o")
#o = u.GetOutput("unfolded","","signal", "*[b]", False)
 

In [31]:
# Get systematic shifts of output
o_up = u.GetDeltaSysSource("Up", "o_upshift", "M+1#sigma")
o_dn = u.GetDeltaSysSource("Dn", "o_dnshift", "M-1#sigma")
o_up.SetLineStyle(2)
o_dn.SetLineStyle(2)


In [32]:
# Get various covariances
cov = u.GetEmatrixTotal("cov", "Covariance Matrix")
print(cov.GetNbinsX())
cov.Draw("colz")
ROOT.gPad.Draw()
#cov = u.GetEmatrixTotal("ematrix","Covariance Matrix", "signal", "*[]", False)

16


In [33]:
cov_uncorr = u.GetEmatrixSysUncorr("cov_uncorr", "Covariance Matrix from Uncorrelated Uncertainties")
cov_uncorr_data = u.GetEmatrixInput("cov_uncorr_data", "Covariance Matrix from Stat Uncertainties of Input Data")

In [34]:
cov_syssource_up = u.GetEmatrixSysUncorr("cov_syssource_up")
cov_syssource_dn = u.GetEmatrixSysUncorr("cov_syssource_dn")
u.GetEmatrixSysSource(cov_syssource_up, "Up")
u.GetEmatrixSysSource(cov_syssource_dn, "Dn")
#cov_uncorr = u.GetEmatrixSysUncorr("cov_uncorr", "Covariance Matrix from Uncorrelated Uncertainties", "signal", "*[]", False)
#cov_uncorr_data = u.GetEmatrixInput("cov_uncorr_data", "Covariance Matrix from Stat Uncertainties of Input Data", "signal", "*[]", False)

In [35]:
cov_syssource_up.Draw("colz")
ROOT.gPad.Draw()

In [36]:
cov_syssource_dn.Draw("colz")
ROOT.gPad.Draw()

In [37]:
cov_uncorr.Draw("colz")
ROOT.gPad.Draw()

In [38]:
cov_uncorr_data.Draw("colz")
ROOT.gPad.Draw()

In [39]:

# Now prepare various distributions. 
o_unfunc = o.Clone("o_unfunc")        # Unfolding and stat unc
o_sys = o.Clone("o_sys")        # Syst uncertainty
o_unctot = o.Clone("o_unctot")  # Total uncertainty
o_sys.SetLineStyle(2)


In [40]:

# Create total uncertainty and sys uncertainty plots. 
# Also fix the uncertainties on the output
o_origErrors = o.Clone("o_origErrors")
o_origJustErrors = o.Clone("o_origErrors")
o_sysErrors = o.Clone("o_sysErrors")

for i in range( 0, o_up.GetNbinsX() + 1):
    unc_tot = ROOT.TMath.Sqrt( cov.GetBinContent(i,i) )
    print( unc_tot )
    o_unctot.SetBinContent(i, unc_tot )   
    
    oe = o.GetBinError(i)
    print(oe)
    o_origJustErrors.SetBinContent(i, oe)    
    o.SetBinError(i, unc_tot )
    
    yup = abs( o_up.GetBinContent(i))
    ydn = abs( o_dn.GetBinContent(i))
    dy = ROOT.TMath.Sqrt( (yup**2 + ydn**2) )
    o_sys.SetBinContent(i, dy )
    o_sysErrors.SetBinError(i, dy )
    print(dy)
    
    dtot = ROOT.TMath.Sqrt(dy**2 + oe**2)
    print(dtot)
    print("")






0.0
0.0
0.0
0.0

0.0
0.0
0.0
0.0

0.0
0.0
0.0
0.0

0.0
0.0
0.0
0.0

0.0
0.0
0.0
0.0

0.0
0.0
0.0
0.0

2975.959352711267
147.95668974604058
2972.111489313355
2975.7919764239327

3041.7716530257467
147.0908690499524
3038.0216163870928
3041.5803565569518

593.5081745973964
67.95976946952224
589.2341578534227
593.1403063758042

0.0
0.0
0.0
0.0

2920.9460216356915
147.92191233845134
2916.996379327483
2920.7445573277214

2874.4814561474846
148.64846629548592
2870.379749906232
2874.2262045990333

558.2667431771849
67.20745461670039
553.7617471670911
557.8251648873553

0.0
0.0
0.0
0.0

5878.857373822113
67.58842740504997
5878.432497758603
5878.821040499833

5859.874696966603
68.4168114330944
5859.428704573176
5859.828120692855

1139.6190595167877
30.901474673604223
1139.1399380848345
1139.5589935044716



In [41]:
#htrue2 = htrue.Clone("h2")
#htrue2.Rebin(finebins/coarsebins)
o.SetMarkerStyle(20)
o.SetMarkerColor(1)
#o_origErrors.SetMarkerStyle(21)
#o_origErrors.SetMarkerColor(3)
o_sysErrors.SetMarkerStyle(21)
o_sysErrors.SetMarkerColor(3)
o_unfunc.SetMarkerStyle(0)
o_unfunc.SetLineColor(2)
htrue.SetMarkerStyle(21)
htrue.SetMarkerColor(4)
htrue.SetMarkerStyle(4)
htrue.SetLineColor(4)

In [42]:
c = ROOT.TCanvas("c","c")
#o.SetMinimum(0.0)
#o.SetMaximum(1.0)

o.Draw()
print (o.GetNbinsX())
#o_origErrors.Draw("e same")
o_sysErrors.Draw("e same")
o_unfunc.Draw("e same")
htrue.Draw("e same")
leg2 = ROOT.TLegend(0.7, 0.7, 0.86, 0.86)
leg2.SetBorderSize(0)
leg2.SetFillColor(0)
leg2.AddEntry( o, "Unfolded, total unc",'p')
leg2.AddEntry( o_origErrors, "Unfolded, unf unc",'p')

leg2.AddEntry( o_unfunc, "stat+unfolding unc",'l')
leg2.AddEntry( htrue, "True",'p')
leg2.Draw()
c.Draw()

16


# Normalized cross section

#### Make Jacobian matrix

In [43]:
print (o.GetNbinsX())

16


In [44]:
J = ROOT.TMatrixD( o.GetNbinsX()+1, o.GetNbinsX()+1 )
ROOT.makeJacobian2Dfrom1D(nmbinsGen+2, nptbinsGen+2, o, J)

In [45]:
J.Draw("colz")
ROOT.gPad.Draw()
#ROOT.gPad.SetLogz()

#### Normalize unfolded output

In [46]:
o.SetTitle("Unfolded Output")
o_norm = o.Clone("o_norm")
o_norm.Draw("hist")
ROOT.gPad.Draw()

In [47]:
for i_ptBins in range (0, nptbinsGen+2):
    print (i_ptBins)
    counter = i_ptBins*(nmbinsGen+2)
    low_bin = 1+counter
    high_bin = counter+nptbinsGen+2
    sum_ptbin = o.Integral(low_bin,high_bin)
    if sum_ptbin > 0.0:
        for j_obins in range(low_bin,high_bin+1):
            o_norm.SetBinContent(j_obins,o.Integral(j_obins,j_obins)/sum_ptbin)

0
1
2
3


In [48]:
o_norm.SetTitle("Normalized unfolded output")
o_norm.Draw("hist")
ROOT.gPad.Draw()

#### Construct covariance TMatrixD for total and individual uncertainties

In [49]:
cov_m = ROOT.TMatrixD( cov.GetNbinsX()+1, cov.GetNbinsY()+1 )
cov_m_up = ROOT.TMatrixD( cov.GetNbinsX()+1, cov.GetNbinsY()+1 )
cov_m_dn = ROOT.TMatrixD( cov.GetNbinsX()+1, cov.GetNbinsY()+1 )
cov_m_uncorr = ROOT.TMatrixD( cov.GetNbinsX()+1, cov.GetNbinsY()+1 )
cov_m_uncorr_data = ROOT.TMatrixD( cov.GetNbinsX()+1, cov.GetNbinsY()+1 )

for xbin in range(0, cov.GetNbinsY()+1 ):
    for ybin in range(0,cov.GetNbinsX()+1 ):
        cov_m[xbin][ybin] = cov.GetBinContent( xbin, ybin)
        cov_m_up[xbin][ybin] = cov_syssource_up.GetBinContent( xbin, ybin)
        cov_m_dn[xbin][ybin] = cov_syssource_dn.GetBinContent( xbin, ybin)
        cov_m_uncorr[xbin][ybin] = cov_uncorr.GetBinContent( xbin, ybin)
        cov_m_uncorr_data[xbin][ybin] = cov_uncorr_data.GetBinContent( xbin, ybin)

In [50]:
cov_m.Draw("colz")
ROOT.gPad.Draw()

In [51]:
cov_m_up.Draw("colz")
ROOT.gPad.Draw()

In [52]:
cov_m_dn.Draw("colz")
ROOT.gPad.Draw()

In [53]:
cov_m_uncorr.Draw("colz")
ROOT.gPad.Draw()

In [54]:
cov_m_uncorr_data.Draw("colz")
ROOT.gPad.Draw()

#### Transform covariance matrices to normalized space

In [55]:
covnorm_temp = ROOT.TMatrixD( cov_m, ROOT.TMatrixD.kMultTranspose, J)
covnorm = ROOT.TMatrixD(J, ROOT.TMatrix.kMult, covnorm_temp )

In [56]:
covnorm_temp_up = ROOT.TMatrixD( cov_m_up, ROOT.TMatrixD.kMultTranspose, J)
covnorm_up = ROOT.TMatrixD(J, ROOT.TMatrix.kMult, covnorm_temp_up )

In [57]:
covnorm_temp_dn = ROOT.TMatrixD( cov_m_dn, ROOT.TMatrixD.kMultTranspose, J)
covnorm_dn = ROOT.TMatrixD(J, ROOT.TMatrix.kMult, covnorm_temp_dn )

In [58]:
covnorm_temp_uncorr_data = ROOT.TMatrixD( cov_m_uncorr_data, ROOT.TMatrixD.kMultTranspose, J)
covnorm_uncorr_data = ROOT.TMatrixD(J, ROOT.TMatrix.kMult, covnorm_temp_uncorr_data )

In [59]:
covnorm_temp_uncorr = ROOT.TMatrixD( cov_m_uncorr, ROOT.TMatrixD.kMultTranspose, J)
covnorm_uncorr = ROOT.TMatrixD(J, ROOT.TMatrix.kMult, covnorm_temp_uncorr )

In [60]:
covnorm.Draw("colz")
ROOT.gPad.Draw()
#ROOT.gPad.SetLogz()

In [61]:
covnorm_up.Draw("colz")
ROOT.gPad.Draw()

In [62]:
covnorm_dn.Draw("colz")
ROOT.gPad.Draw()

In [63]:
covnorm_uncorr.Draw("colz")
ROOT.gPad.Draw()

In [64]:
covnorm_uncorr_data.Draw("colz")
ROOT.gPad.Draw()

In [65]:
covnorm_sum = covnorm_up.Clone("covnorm_sum")
covnorm_sum += covnorm_dn
covnorm_sum += covnorm_uncorr
covnorm_sum += covnorm_uncorr_data
covnorm_sum.Draw("colz")
ROOT.gPad.Draw()

#### Now prepare various distributions

In [66]:
#o_unfunc_norm = o_unfunc.Clone("o_unc_norm")          # Unfolding and stat unc
#o_sys_norm = o_norm.Clone("o_sys_norm")               # Syst uncertainty
o_unctot_norm = o_norm.Clone("o_unctot_norm")          # Total uncertainty
o_unc_up_norm = o_norm.Clone("o_unc_up_norm")          # Up uncertainty
o_unc_dn_norm = o_norm.Clone("o_unc_dn_norm")          # Down uncertainty
o_unc_data_norm = o_norm.Clone("o_unc_data_norm")      # Input data stat uncertainty
o_unc_uncorr_norm = o_norm.Clone("o_unc_uncorr_norm")  # Uncorrelated uncertainties
o_unc_sum_norm = o_norm.Clone("o_sum_data_norm")       # Sum of uncertainty sources
#o_sys_norm.SetLineStyle(2)

In [67]:
#o_unctot_norm.Draw("hist")
#ROOT.gPad.Draw()

#### Create total uncertainty and sys uncertainty plots
#### Set correct uncertainty for normalized unfolded output

In [80]:
# Create total uncertainty and sys uncertainty plots. 
# Set correct uncertainty for normalized unfolded output
for i in range( 0, o.GetNbinsX() + 1):
    unc_tot_norm = ROOT.TMath.Sqrt( covnorm[i][i] )
    unc_up_norm = ROOT.TMath.Sqrt( covnorm_up[i][i] )
    unc_dn_norm = ROOT.TMath.Sqrt( covnorm_dn[i][i] )
    unc_data_norm = ROOT.TMath.Sqrt( covnorm_uncorr_data[i][i] )
    unc_uncorr_norm = ROOT.TMath.Sqrt( covnorm_uncorr[i][i] )
    unc_sum_norm = ROOT.TMath.Sqrt( covnorm_sum[i][i] )
    
    print(unc_tot_norm)
    print(unc_up_norm)
    print(unc_dn_norm)
    print(unc_data_norm)
    print(unc_uncorr_norm)
    print(unc_sum_norm)
    print("")
    
    o_unctot_norm.SetBinContent(i, unc_tot_norm )
    o_unc_up_norm.SetBinContent(i, unc_up_norm )
    o_unc_dn_norm.SetBinContent(i, unc_dn_norm )
    o_unc_data_norm.SetBinContent(i, unc_data_norm )
    o_unc_uncorr_norm.SetBinContent(i, unc_uncorr_norm )
    o_unc_sum_norm.SetBinContent(i, unc_sum_norm  )
    
    o_norm.SetBinError(i, unc_tot_norm )

0.0
0.0
0.0
0.0
0.0
0.0

0.0
0.0
0.0
0.0
0.0
0.0

0.0
0.0
0.0
0.0
0.0
0.0

0.0
0.0
0.0
0.0
0.0
0.0

0.0
0.0
0.0
0.0
0.0
0.0

0.0
0.0
0.0
0.0
0.0
0.0

0.0028972855726681376
5.3564846722917716e-05
0.0015227775061681821
0.002400391333128285
0.000557372955333478
0.0028972855726681133

0.0031030618011213666
0.0003238138560790467
0.0017734616719455895
0.0024301713460710547
0.0006879230008255237
0.0031030618011213414

0.0015430344219206852
0.0002702490093561057
0.00025068416577734567
0.0014275349573112633
0.0004552165362297303
0.0015430344219206854

0.0
0.0
0.0
0.0
0.0
0.0

0.002583370994825818
0.0006643660254166526
2.8192476302622838e-05
0.002418850533882657
0.0006170824581811211
0.002583370994825804

0.0025860182671553628
0.00010536346586402731
0.00031608436316649607
0.002453904286448187
0.0007448714294663629
0.002586018267155377

0.0016266144846010546
0.0005590025595533276
0.00028789188686387647
0.0014200313153983183
0.00048375628646338915
0.0016266144846010504

0.0
0.0
0.0
0.0
0.0
0.0

0.

In [81]:
o_norm.SetTitle("Normalized unfolded results")
o_norm.Draw("EP")
ROOT.gPad.Draw()

In [82]:
o_unctot_norm.SetTitle("Normalized unfolded uncertainties")

o_unctot_norm.SetLineColor(ROOT.kBlack)
o_unctot_norm.SetLineWidth(3)
o_unc_up_norm.SetLineColor(ROOT.kGreen+1)
o_unc_dn_norm.SetLineColor(ROOT.kRed)
o_unc_data_norm.SetLineColor(ROOT.kBlue)
o_unc_uncorr_norm.SetLineColor(ROOT.kCyan+1)
o_unc_sum_norm.SetLineColor(ROOT.kMagenta+1)

c_err = ROOT.TCanvas("c_err","c_err")

o_unctot_norm.Draw("hist")
o_unc_up_norm.Draw("hist same")
o_unc_dn_norm.Draw("hist same")
o_unc_data_norm.Draw("hist same")
o_unc_uncorr_norm.Draw("hist same")
o_unc_sum_norm.Draw("hist same")

leg_err = ROOT.TLegend(0.7, 0.7, 0.86, 0.86)
leg_err.SetBorderSize(0)
leg_err.SetFillColor(0)
leg_err.AddEntry( o_unctot_norm, "Total",'l')
leg_err.AddEntry( o_unc_up_norm, "Up",'l')
leg_err.AddEntry( o_unc_dn_norm, "Down",'l')
leg_err.AddEntry( o_unc_data_norm, "Data Stat",'l')
leg_err.AddEntry( o_unc_uncorr_norm, "Uncorr",'l')
leg_err.AddEntry( o_unc_sum_norm, "Sum of Uncerts",'l')
leg_err.Draw()
c_err.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_err


In [83]:
o_unctot_norm_rel = o_unctot_norm.Clone("o_unctot_norm_rel")
o_unc_up_norm_rel = o_unc_up_norm.Clone("o_unc_up_rel")
o_unc_dn_norm_rel = o_unc_dn_norm.Clone("o_unc_dn_rel")
o_unc_data_norm_rel = o_unc_data_norm.Clone("o_unc_data_norm_rel")
o_unc_uncorr_norm_rel = o_unc_uncorr_norm.Clone("o_unc_uncorr_norm_rel")
o_unc_sum_norm_rel = o_unc_sum_norm.Clone("o_unc_sum_rel")

In [84]:
o_unctot_norm_rel.Divide(o_norm)
o_unc_up_norm_rel.Divide(o_norm)
o_unc_dn_norm_rel.Divide(o_norm)
o_unc_data_norm_rel.Divide(o_norm)
o_unc_uncorr_norm_rel.Divide(o_norm)
o_unc_sum_norm_rel.Divide(o_norm)

True

In [85]:
o_unctot_norm_rel.SetTitle("Normalized fractional unfolded uncertainties")

c_err_rel = ROOT.TCanvas("c_err_rel","c_err_rel")

o_unctot_norm_rel.Draw("hist")
o_unc_up_norm_rel.Draw("hist same")
o_unc_dn_norm_rel.Draw("hist same")
o_unc_data_norm_rel.Draw("hist same")
o_unc_uncorr_norm_rel.Draw("hist same")
o_unc_sum_norm_rel.Draw("hist same")

leg_err_rel = ROOT.TLegend(0.7, 0.7, 0.86, 0.86)
leg_err_rel.SetBorderSize(0)
leg_err_rel.SetFillColor(0)
leg_err_rel.AddEntry( o_unctot_norm_rel, "Total",'l')
leg_err_rel.AddEntry( o_unc_up_norm_rel, "Up",'l')
leg_err_rel.AddEntry( o_unc_dn_norm_rel, "Down",'l')
leg_err_rel.AddEntry( o_unc_data_norm_rel, "Data Stat",'l')
leg_err_rel.AddEntry( o_unc_uncorr_norm_rel, "Uncorr",'l')
leg_err_rel.AddEntry( o_unc_sum_norm_rel, "Sum of Uncerts",'l')
leg_err_rel.Draw()
c_err_rel.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_err_rel


## Plot closure

The closure plot is the true distribution divided by the unfolded distribution. Aside from the outermost bins, the closure is consistent with unity. 

In [74]:
'''c2 = ROOT.TCanvas("c2", "c2", 500, 500)
closure = htrue.Clone("closure")
closure2 = htrue.Clone("closure2")
closure.SetMarkerColor(1)
closure.SetLineColor(1)
closure2.SetMarkerColor(2)
closure2.SetLineColor(2)
closure.SetTitle("true / unfolded")
closure.Divide(o)
closure2.Divide(o_unfunc)
closure.Draw('e1')
closure2.Draw('e1 same')
closure.SetMaximum(4.1)
closure.SetMinimum(-1.1)

legclosure = ROOT.TLegend(0.4, 0.7, 0.86, 0.86)
legclosure.SetBorderSize(0)
legclosure.SetFillColor(0)
legclosure.AddEntry( closure, "True / Unfolded, All Unc.", 'p')
legclosure.AddEntry( closure2, "True / Unfolded, Stat. Unc.", 'p')
legclosure.Draw()
c2.Draw()'''

'c2 = ROOT.TCanvas("c2", "c2", 500, 500)\nclosure = htrue.Clone("closure")\nclosure2 = htrue.Clone("closure2")\nclosure.SetMarkerColor(1)\nclosure.SetLineColor(1)\nclosure2.SetMarkerColor(2)\nclosure2.SetLineColor(2)\nclosure.SetTitle("true / unfolded")\nclosure.Divide(o)\nclosure2.Divide(o_unfunc)\nclosure.Draw(\'e1\')\nclosure2.Draw(\'e1 same\')\nclosure.SetMaximum(4.1)\nclosure.SetMinimum(-1.1)\n\nlegclosure = ROOT.TLegend(0.4, 0.7, 0.86, 0.86)\nlegclosure.SetBorderSize(0)\nlegclosure.SetFillColor(0)\nlegclosure.AddEntry( closure, "True / Unfolded, All Unc.", \'p\')\nlegclosure.AddEntry( closure2, "True / Unfolded, Stat. Unc.", \'p\')\nlegclosure.Draw()\nc2.Draw()'

## Plot the total covariance matrix

In [75]:
'''ccov = ROOT.TCanvas("ccov", "Covariance Matrix", 400, 400)
cov.Draw('colz')
ccov.SetLogz()
ccov.Draw()'''


'ccov = ROOT.TCanvas("ccov", "Covariance Matrix", 400, 400)\ncov.Draw(\'colz\')\nccov.SetLogz()\nccov.Draw()'

## Plot the covariance matrix from uncorrelated (stat.) uncertainties

In [76]:
'''ccov_uncorr = ROOT.TCanvas("ccov_uncorr", "Covariance Matrix from Uncorrelated Uncertainties", 400, 400)
cov_uncorr.Draw('colz')
ccov_uncorr.SetLogz()
ccov_uncorr.Draw()'''

'ccov_uncorr = ROOT.TCanvas("ccov_uncorr", "Covariance Matrix from Uncorrelated Uncertainties", 400, 400)\ncov_uncorr.Draw(\'colz\')\nccov_uncorr.SetLogz()\nccov_uncorr.Draw()'

## Plot covariance matrix of the statistical uncertainties from the input data

In [77]:
'''ccov_uncorr_data = ROOT.TCanvas("ccov_uncorr_data", "Covariance Matrix from Uncorrelated Uncertainties", 400, 400)
cov_uncorr_data.Draw('colz')
ccov_uncorr_data.SetLogz()
ccov_uncorr_data.Draw()'''

'ccov_uncorr_data = ROOT.TCanvas("ccov_uncorr_data", "Covariance Matrix from Uncorrelated Uncertainties", 400, 400)\ncov_uncorr_data.Draw(\'colz\')\nccov_uncorr_data.SetLogz()\nccov_uncorr_data.Draw()'

## Plot fractional uncertainties



In [78]:
'''o_fracUnc = o_sys.Clone()
o_fracUnc.Divide(o.Clone())

o_fracUnc.SetTitle("Fractional Uncertainty")

c_sysf = ROOT.TCanvas("c_sys", "c_sys", 500, 500)
o_fracUnc.Draw("e")

o_fracUnc.SetMinimum(0.)
o_fracUnc.SetMaximum(.4)

legsysf = ROOT.TLegend(0.7, 0.7, 0.86, 0.86)
legsysf.SetFillColor(0)
legsysf.SetBorderSize(0)

legsysf.AddEntry( o_fracUnc, 'Syst', 'l')
legsysf.Draw()
c_sysf.Draw()'''

'o_fracUnc = o_sys.Clone()\no_fracUnc.Divide(o.Clone())\n\no_fracUnc.SetTitle("Fractional Uncertainty")\n\nc_sysf = ROOT.TCanvas("c_sys", "c_sys", 500, 500)\no_fracUnc.Draw("e")\n\no_fracUnc.SetMinimum(0.)\no_fracUnc.SetMaximum(.4)\n\nlegsysf = ROOT.TLegend(0.7, 0.7, 0.86, 0.86)\nlegsysf.SetFillColor(0)\nlegsysf.SetBorderSize(0)\n\nlegsysf.AddEntry( o_fracUnc, \'Syst\', \'l\')\nlegsysf.Draw()\nc_sysf.Draw()'

## Plot absolute uncertainties


In [79]:
'''unc_stack = ROOT.THStack("unc_stack", "Uncertainties")
unc_stack.Add( o_unctot, "hist" )
unc_stack.Add( o_sys, "hist" )
#o_up.Divide(o)
#o_dn.Divide(o)



c_sys = ROOT.TCanvas("c_sys", "c_sys", 500, 500)
unc_stack.Draw("nostack")
unc_stack.SetMaximum( 1.0 * unc_stack.GetMaximum() )
#o_unc.SetMinimum(-0.2)
#o_unc.SetMaximum(+0.2)
legsys = ROOT.TLegend(0.7, 0.7, 0.86, 0.86)
legsys.SetFillColor(0)
legsys.SetBorderSize(0)
legsys.AddEntry( o_unctot, 'Total', 'l')
legsys.AddEntry( o_sys, 'Syst', 'l')
legsys.Draw()
c_sys.Draw()'''

'unc_stack = ROOT.THStack("unc_stack", "Uncertainties")\nunc_stack.Add( o_unctot, "hist" )\nunc_stack.Add( o_sys, "hist" )\n#o_up.Divide(o)\n#o_dn.Divide(o)\n\n\n\nc_sys = ROOT.TCanvas("c_sys", "c_sys", 500, 500)\nunc_stack.Draw("nostack")\nunc_stack.SetMaximum( 1.0 * unc_stack.GetMaximum() )\n#o_unc.SetMinimum(-0.2)\n#o_unc.SetMaximum(+0.2)\nlegsys = ROOT.TLegend(0.7, 0.7, 0.86, 0.86)\nlegsys.SetFillColor(0)\nlegsys.SetBorderSize(0)\nlegsys.AddEntry( o_unctot, \'Total\', \'l\')\nlegsys.AddEntry( o_sys, \'Syst\', \'l\')\nlegsys.Draw()\nc_sys.Draw()'